In [ ]:
pip install torch torchvision torchaudio yolov5 matplotlib shutil

In [ ]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from yolov5 import YOLOv5

In [ ]:
pretrained_file_path = '../yolov5m.pt'
model = YOLOv5(pretrained_file_path)

if os.path.exists(pretrained_file_path):
    model = YOLOv5(pretrained_file_path)
    print("Model loaded successfully.")
else:
    raise FileNotFoundError(f"File {pretrained_file_path} not found.")

In [ ]:
# Define source and destination directories
src_dir = '../data/cam1'
dst_dir = '../data/cam1_rgb'

# Create destination directory if it doesn't exist
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Function to delete a directory if it exists
def delete_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)

# Function to recreate a directory
def recreate_directory(directory):
    os.makedirs(directory)

# Function to process images
def process_images(src_dir, dst_dir):
    for filename in os.listdir(src_dir):
        if filename.endswith('.png'):
            # Read the image
            image_path = os.path.join(src_dir, filename)
            image = cv2.imread(image_path)
            
            # Ensure the image is loaded correctly
            if image is not None:
                # Get the right half of the image (640x928)
                height, width = image.shape[:2]
                right_half = image[:, width//2:]

                # Save the right half to the destination directory
                dst_image_path = os.path.join(dst_dir, filename)
                cv2.imwrite(dst_image_path, right_half)
            else:
                print(f"Failed to load image: {image_path}")

# Process the images
process_images(src_dir, dst_dir)

print(f"Processing complete. The right half of the images have been saved to {dst_dir}")

In [ ]:
# Define the directory containing the JSON files
annotation_dir = '../data/cam1_labeled'

# List all files in the annotation directory
files = os.listdir(annotation_dir)

# Loop through the files and rename JSON files
for filename in files:
    if filename.endswith('_json.json'):
        new_filename = filename.replace('_json', '')
        old_filepath = os.path.join(annotation_dir, filename)
        new_filepath = os.path.join(annotation_dir, new_filename)
        os.rename(old_filepath, new_filepath)
        print(f'Renamed: {filename} to {new_filename}')

In [ ]:
import json
import os
from PIL import Image, ImageDraw, ImageOps

def resize_image(image, target_size):
    # Check if the image dimensions match the target size
    if image.size == target_size:
        return image
    else:
        return image.resize(target_size)

def segment_image(json_file, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    for filename, image_data in data.items():
        # Construct paths for segmented image and original image
        original_img_path = os.path.join('../data/cam1_rgb', filename)
        
        # Check if the original image file exists
        if not os.path.exists(original_img_path):
            print(f"Error: Original image file '{original_img_path}' not found.")
            continue
        
        # Open original image
        original_image = Image.open(original_img_path)
        
        # Resize image if needed
        target_size = (640, 928)
        resized_image = resize_image(original_image, target_size)
        
        # Create a mask image for the regions to remove
        mask = Image.new('L', resized_image.size, 0)
        draw = ImageDraw.Draw(mask)
        
        regions_to_remove = []
        regions = image_data.get('regions', [])
        for idx, region in enumerate(regions):
            region_attributes = region.get('region_attributes', {})
            if region_attributes:
                # If region_attributes is empty, mark for removal
                shape_attributes = region.get('shape_attributes', {})
                x = shape_attributes.get('x', 0)
                y = shape_attributes.get('y', 0)
                width = shape_attributes.get('width', 0)
                height = shape_attributes.get('height', 0)
                
                # Draw rectangle on mask
                draw.rectangle([x, y, x + width, y + height], fill=255)
                regions_to_remove.append(idx)
        
        # Invert the mask to remove everything except the segmented region
        inverted_mask = ImageOps.invert(mask)
        
        # Apply the inverted mask to segment the resized image
        segmented_image = Image.composite(resized_image, Image.new('RGB', resized_image.size, (255, 255, 255)), inverted_mask)
        
        # Save the segmented image with a new filename
        new_filename = f"segmented_{os.path.splitext(filename)[0]}.png"
        segmented_img_path = os.path.join(output_folder, new_filename)
        segmented_image.save(segmented_img_path)
        print(f"Saved segmented image: {segmented_img_path}")

# Example usage
json_file_path = '../data/cam1_test.json'          # Replace with your JSON file path
output_folder = '../data/cam1_rgb/'                # Replace with your output folder path

# Perform segmentation
segment_image(json_file_path, output_folder)

### After Use, delete the directories

In [ ]:
delete_directory('../data/cam1_rgb')

# delete_directory('../data/cam1_labeled')